In [46]:
#Import and Treat Data
import pandas as pd
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
#VIF
from sklearn.linear_model import LinearRegression
from collections import deque
import datetime
import pickle
#NN
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
import torch.optim as optim

# Os dados

### Importando o Dataset e transformando em variável:

In [47]:
df = pd.read_csv('dataset.csv', delimiter = ';')
df = df.dropna()
df

,ID,T,tau,time,N_total,N_bulk,N_surface,Volume,R_min,R_max,...,q6q6_S14,q6q6_S15,q6q6_S16,q6q6_S17,q6q6_S18,q6q6_S19,q6q6_S20,q6q6_S20+,Total_E,Formation_E
0,3281,923,45,1,133,42,91,1.570000e-27,4.737965,8.103804,...,0,0,0,0,0,0,0,0,-385.20936,85.61064
1,3282,923,45,2,136,44,92,1.600000e-27,4.023433,8.361129,...,0,0,0,0,0,0,0,0,-393.45040,87.98960
2,2291,723,50,1,149,51,98,1.750000e-27,4.571391,7.953961,...,0,0,0,0,0,0,0,0,-441.89002,85.56998
3,2292,723,50,2,151,54,97,1.780000e-27,4.754535,8.400565,...,0,0,0,0,0,0,0,0,-451.64457,82.89543
4,3091,823,50,1,170,67,103,2.000000e-27,5.673022,8.205280,...,0,0,0,0,0,0,0,0,-506.59469,95.20531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,1210,523,5,10,19487,16311,3176,2.290000e-25,34.466223,44.887615,...,0,0,0,0,0,0,0,0,-65609.94800,3374.03200
3996,2110,673,5,10,19507,16289,3218,2.300000e-25,34.247301,44.673954,...,0,0,0,0,0,0,0,0,-65392.19100,3662.58900
3997,1010,423,5,10,19509,16181,3328,2.300000e-25,21.874439,46.705838,...,0,0,0,0,0,0,0,0,-66085.15400,2976.70600
3998,1910,573,5,10,19517,16255,3262,2.300000e-25,33.916720,48.546207,...,0,0,0,0,0,0,0,0,-65662.22800,3427.95200


### Explorando o Dataset

### Separando em treino e teste

[1]

In [48]:
#Definição do que serão as features e os targets
X = df[["T", "tau", "time", "N_total", "N_bulk", "N_surface", "Volume", "R_min", "R_max", "R_diff", "R_avg", "R_std", "R_skew", "R_kurt", "S_100", "S_111", "S_110", "S_311", "Curve_1-10", "Curve_11-20", "Curve_21-30", "Curve_31-40", "Curve_41-50", "Curve_51-60", "Curve_61-70", "Curve_71-80", "Curve_81-90", "Curve_171-180", "Avg_total", "Avg_bulk", "Avg_surf", "TCN_1", "TCN_2", "TCN_3", "TCN_4", "TCN_5", "TCN_6", "TCN_7", "TCN_8", "TCN_9", "TCN_10", "TCN_11", "TCN_12", "TCN_13", "TCN_14", "TCN_15", "TCN_16", "BCN_6", "BCN_7", "BCN_8", "BCN_9", "BCN_10", "BCN_11", "BCN_12", "BCN_13", "BCN_14", "BCN_15", "BCN_16", "SCN_1", "SCN_2", "SCN_3", "SCN_4", "SCN_5", "SCN_6", "SCN_7", "SCN_8", "SCN_9", "SCN_10", "SCN_11", "SCN_12", "SCN_13", "SCN_14", "Avg_bonds", "Std_bonds", "Max_bonds", "Min_bonds", "N_bonds", "angle_avg", "angle_std", "FCC", "HCP", "ICOS", "DECA", "q6q6_avg_total", "q6q6_avg_bulk", "q6q6_avg_surf", "q6q6_T0", "q6q6_T1", "q6q6_T2", "q6q6_T3", "q6q6_T4", "q6q6_T5", "q6q6_T6", "q6q6_T7", "q6q6_T8", "q6q6_T9", "q6q6_T10", "q6q6_T11", "q6q6_T12", "q6q6_T13", "q6q6_T14", "q6q6_B0", "q6q6_B1", "q6q6_B2", "q6q6_B3", "q6q6_B4", "q6q6_B5", "q6q6_B6", "q6q6_B7", "q6q6_B8", "q6q6_B9", "q6q6_B10", "q6q6_B11", "q6q6_B12", "q6q6_B13", "q6q6_B14", "q6q6_B15", "q6q6_B16", "q6q6_B17", "q6q6_B18", "q6q6_B19", "q6q6_B20", "q6q6_B20+", "q6q6_S0", "q6q6_S1", "q6q6_S2", "q6q6_S3", "q6q6_S4", "q6q6_S5", "q6q6_S6", "q6q6_S7", "q6q6_S8", "q6q6_S9", "q6q6_S10", "q6q6_S11", "q6q6_S12", "q6q6_S13", "q6q6_S20+"]]
y = df[["Total_E", "Formation_E"]]
atributos = ["T", "tau", "time", "N_total", "N_bulk", "N_surface", "Volume", "R_min", "R_max", "R_diff", "R_avg", "R_std", "R_skew", "R_kurt", "S_100", "S_111", "S_110", "S_311", "Curve_1-10", "Curve_11-20", "Curve_21-30", "Curve_31-40", "Curve_41-50", "Curve_51-60", "Curve_61-70", "Curve_71-80", "Curve_81-90", "Curve_171-180", "Avg_total", "Avg_bulk", "Avg_surf", "TCN_1", "TCN_2", "TCN_3", "TCN_4", "TCN_5", "TCN_6", "TCN_7", "TCN_8", "TCN_9", "TCN_10", "TCN_11", "TCN_12", "TCN_13", "TCN_14", "TCN_15", "TCN_16", "BCN_6", "BCN_7", "BCN_8", "BCN_9", "BCN_10", "BCN_11", "BCN_12", "BCN_13", "BCN_14", "BCN_15", "BCN_16", "SCN_1", "SCN_2", "SCN_3", "SCN_4", "SCN_5", "SCN_6", "SCN_7", "SCN_8", "SCN_9", "SCN_10", "SCN_11", "SCN_12", "SCN_13", "SCN_14", "Avg_bonds", "Std_bonds", "Max_bonds", "Min_bonds", "N_bonds", "angle_avg", "angle_std", "FCC", "HCP", "ICOS", "DECA", "q6q6_avg_total", "q6q6_avg_bulk", "q6q6_avg_surf", "q6q6_T0", "q6q6_T1", "q6q6_T2", "q6q6_T3", "q6q6_T4", "q6q6_T5", "q6q6_T6", "q6q6_T7", "q6q6_T8", "q6q6_T9", "q6q6_T10", "q6q6_T11", "q6q6_T12", "q6q6_T13", "q6q6_T14", "q6q6_B0", "q6q6_B1", "q6q6_B2", "q6q6_B3", "q6q6_B4", "q6q6_B5", "q6q6_B6", "q6q6_B7", "q6q6_B8", "q6q6_B9", "q6q6_B10", "q6q6_B11", "q6q6_B12", "q6q6_B13", "q6q6_B14", "q6q6_B15", "q6q6_B16", "q6q6_B17", "q6q6_B18", "q6q6_B19", "q6q6_B20", "q6q6_B20+", "q6q6_S0", "q6q6_S1", "q6q6_S2", "q6q6_S3", "q6q6_S4", "q6q6_S5", "q6q6_S6", "q6q6_S7", "q6q6_S8", "q6q6_S9", "q6q6_S10", "q6q6_S11", "q6q6_S12", "q6q6_S13", "q6q6_S20+"]

# Split de treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.1, random_state=10)
y_treino = y_treino.dropna()
X_treino = X_treino.dropna()

In [49]:
X = torch.tensor(np.array(y_treino))
X.size(0)

3600

### Aplicando o Logaritmo e Normalização pelo máximo absoluto

In [50]:
#X_treino_log = np.log(np.array(X_treino))
#X_teste_log = np.log(np.array(X_teste))
#X_treino_log = X_treino_log.dropna()

In [51]:
#[2]
max_abs_normalizador = MaxAbsScaler()
X_treino_log_normalizado = max_abs_normalizador.fit_transform(X_treino)
X_teste_log_normalizado = max_abs_normalizador.fit_transform(X_teste)

In [52]:
x = torch.tensor(np.array(Y_treino))
x.size(0)

NameError: name 'Y_treino' is not defined

### Aplicando o VIF

In [ ]:
def vif_selection(
    x: np.ndarray,
    cols: list,
    max_vif: float = 8,
    deleted_cols=[],
    savepath=None,
    verbose=False,
    initial_pass=False,
):

    min_tol = 1 / max_vif
    idx = 0

    def quick_tolerance_min_idx(x, idx=0):
        tolerance = np.zeros(x.shape[1])

        # to improve speed of finding the intial zeros
        gen = deque(range(x.shape[1]))
        gen.rotate(-idx)
        gen = list(gen)

        for i in gen:
            X, y = np.delete(x, i, 1), x[:, i]
            # https://stackoverflow.com/questions/36573046/difference-between-numpy-linalg-lstsq-and-sklearn-linear-model-linearregression
            r_squared = LinearRegression().fit(X, y).score(X, y)
            tol = 1 - r_squared
            if tol == 0:
                return i, tol
            else:
                tolerance[i] = tol

        idx = np.argmin(tolerance)
        return idx, tolerance[idx]

    if verbose:
        print(len(cols), datetime.datetime.now())

    def check(tol):
        if initial_pass:
            return tol == 0
        else:
            return tol < min_tol

    while True:
        idx, tol = quick_tolerance_min_idx(x, idx)
        if check(tol):
            x = np.delete(x, idx, 1)
            poped_col = cols.pop(idx)
            deleted_cols.extend([poped_col])
            if savepath:
                pickle.dump(deleted_cols, open(savepath, "wb"))

            if verbose:
                print(
                    len(cols), datetime.datetime.now(), f"{tol:.3g}", poped_col
                )
        else:
            break

    return x, cols, deleted_cols

In [ ]:
X_norm_vif = vif_selection(X_treino_log_normalizado, atributos)

In [ ]:
X_final, cols, deleted_cols = X_norm_vif
cols

# A MLP

In [ ]:
class MLP(nn.Module):

    def __init__(self, num_dados_entrada, neuronios_camadas, num_targets):
        super().__init__()
        
        # Lista para armazenar as camadas
        layers = []
        
        # Adicionando camadas intermediárias
        for i in range(len(neuronios_camadas) - 1):
            layers.append(nn.Linear(neuronios_camadas[i], neuronios_camadas[i+1]))
            layers.append(nn.Sigmoid())
        
        # Adicionando camada de saída
        layers.append(nn.Linear(neuronios_camadas[-1], num_targets))
        
        # Definindo as camadas como uma sequência
        self.camadas = nn.Sequential(*layers)
        
    def forward(self, x):
        x = x.float()
        x = self.camadas(x)
        return x
    
x = torch.tensor(X_final)
y = torch.tensor(np.array(y_treino))
y = y.float()

In [ ]:
y.type()

In [ ]:
x.shape[1]

In [ ]:
NUM_DADOS_DE_ENTRADA = x.shape[1]  
NUM_DADOS_DE_SAIDA = 2  
NEURONIOS = list(range(10, 31, 2))
CAMADAS = [2, 3, 4, 5]


for layer in CAMADAS:
    
    for neuronios in NEURONIOS:
        neuronios_camadas = [NUM_DADOS_DE_ENTRADA]
        
        for a in range(layer):
            neuronios_camadas.append(a)
            
        minha_mlp = MLP(
            NUM_DADOS_DE_ENTRADA, neuronios_camadas, NUM_DADOS_DE_SAIDA
        )
        
        TAXA_DE_APRENDIZADO = 0.001
        otimizador = optim.SGD(minha_mlp.parameters(), lr=TAXA_DE_APRENDIZADO)
        
        fn_perda = nn.MSELoss()
        
        NUM_EPOCAS = 10000
        minha_mlp.train()

        for epoca in range(NUM_EPOCAS):
            # forward pass
            y_pred = minha_mlp(x)

            # zero grad
            otimizador.zero_grad()

            # loss
            loss = fn_perda(y, y_pred)

            # backpropagation
            loss.backward()

            # atualiza parâmetros
            otimizador.step()

            # mostra resultado (opcional)
            print(epoca, loss.data)

# Referências

[1] Caderno da disciplina de Redes Neurais da Ilum Escola De Ciências (ATP - 303 NN 5.3 - PyTorch Lightning)
[2] https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MaxAbsScaler.html